<a href="https://colab.research.google.com/github/anabeatrizzdm/moviesystem/blob/main/moviesystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
caminho_csv1 = '/content/drive/My Drive/moviesystem/movies.csv'

In [ ]:
caminho_csv2 = '/content/drive/My Drive/moviesystem/credits.csv'

In [ ]:
credits = pd.read_csv(caminho_csv2)

In [ ]:
movies = pd.read_csv(caminho_csv1)

In [ ]:
print(movies.head())

In [ ]:
print(credits.head())

In [ ]:
print('credits', credits.shape)
print('movies', movies.shape)

In [ ]:
credits_column_renamed = credits.rename(index=str, columns={"movie_id": "id"})
movies_merged= movies.merge(credits_column_renamed, on="id")
movies_merged.head()

In [ ]:
movies_cleaned = movies_merged.drop(columns=['title_x', 'vote_count', 'title_y', 'homepage', 'keywords', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'cast', 'crew'])
movies_cleaned.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3, max_features= None,
                      strip_accents = 'unicode', analyzer= 'word', token_pattern=r'\w{1,}',
                      ngram_range=(1,3),
                      stop_words = 'english' )

movies_cleaned['overview']= movies_cleaned['overview'].fillna('')

In [ ]:
tfv_matrix = tfv.fit_transform(movies_cleaned['overview'])

In [ ]:
tfv_matrix

In [ ]:
tfv_matrix.shape

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel

In [ ]:
sig = sigmoid_kernel(tfv_matrix,tfv_matrix)

In [ ]:
sig[0]

In [ ]:
indices= pd.Series(movies_cleaned.index, index=movies_cleaned['original_title']).drop_duplicates()

In [ ]:
indices

In [ ]:
def give_recommendation(title, sig=sig):
  idx = indices[title]
  sig_scores = list(enumerate(sig[idx]))
  sig_scores = sorted(sig_scores, key=lambda x:x[1], reverse=True)
  sig_scores = sig_scores[1:8]
  movie_indices = [i[0] for i in sig_scores]
  return movies_cleaned['original_title'].iloc[movie_indices]

In [ ]:
give_recommendation('Creed')

4333                                   Rocky
1960                            Rocky Balboa
2205                              Undisputed
2917                             The Fighter
503     The Adventures of Rocky & Bullwinkle
2273                          Hands of Stone
3037         The Goods: Live Hard, Sell Hard
Name: original_title, dtype: object